In [1]:
from utils import *
import os.path as osp
from transformers import logging
from hparam_tuning import *

logging.set_verbosity_warning()


# Models
## Model Types

In this work we experiment with 3 model types of different sized:
- RoBERTa (base, large)
- ELECTRA (small, base, large)
- DeBERTa (base)

In [8]:
model_names = [
         "google/electra-small-discriminator",
         "google/electra-base-discriminator",
#          "FacebookAI/roberta-base",
         "microsoft/deberta-base",
         "FacebookAI/roberta-large",
         "google/electra-large-discriminator",
]

## Hyperparameters Tuning

In [3]:
for model_name in model_names:
    dataset = load_csv_to_dataset("data/train.csv", "data/validation.csv")    
    tune_hyperparams(model_name=model_name, dataset=dataset, weighted=True)

Loading best hyperparameters for google/electra-small-discriminator from hparams/google/electra-small-discriminator.json
Loading best hyperparameters for google/electra-base-discriminator from hparams/google/electra-base-discriminator.json
Loading best hyperparameters for FacebookAI/roberta-base from hparams/FacebookAI/roberta-base.json
Loading best hyperparameters for microsoft/deberta-base from hparams/microsoft/deberta-base.json
Loading best hyperparameters for FacebookAI/roberta-large from hparams/FacebookAI/roberta-large.json
Loading best hyperparameters for google/electra-large-discriminator from hparams/google/electra-large-discriminator.json


## Training

In [35]:
dataset = load_csv_to_dataset("data/train.csv", "data/validation.csv")
for model_name in model_names:
    hparams = load_hparams(model_name)
    train(model_name, dataset, ckpt_dir, batch_size=32, log_results=True, trained_weights_dir="trained_weights", **hparams)

Loading best hyperparameters for google/electra-small-discriminator from hparams/google/electra-small-discriminator.json


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,1.383500,0.775803,0.661563,0.771532,0.681838,0.674080
2,0.533100,0.310296,0.913750,0.923337,0.876761,0.894827
3,0.282700,0.238596,0.926250,0.930382,0.883943,0.902969
4,0.215000,0.204933,0.928438,0.929510,0.884113,0.903480
5,0.162900,0.198389,0.930625,0.933634,0.891007,0.907111
6,0.141000,0.203139,0.931562,0.927743,0.890944,0.904584
7,0.121600,0.192881,0.934063,0.929759,0.894031,0.908785
8,0.101600,0.212273,0.932187,0.929987,0.888108,0.904507
9,0.082300,0.213788,0.933125,0.932729,0.891651,0.908827


Loading best hyperparameters for google/electra-base-discriminator from hparams/google/electra-base-discriminator.json


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

Loading best hyperparameters for google/electra-large-discriminator from hparams/google/electra-large-discriminator.json


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,1.760500,1.478120,0.415938,0.306467,0.255583,0.210137
2,0.726900,0.377687,0.912188,0.907284,0.875628,0.886115
3,0.324100,0.323646,0.907188,0.909267,0.857033,0.877074
4,0.240100,0.258820,0.925937,0.931211,0.879580,0.900235
5,0.190800,0.203004,0.935937,0.908391,0.909138,0.905436
6,0.148300,0.177262,0.935000,0.941996,0.892537,0.910907
7,0.138200,0.187960,0.935000,0.946346,0.887937,0.911002
8,0.137700,0.190742,0.936875,0.943161,0.887290,0.910017


/media/embedded/Users/Nisim/MSc/NLP/NLP-using-DL-Transformers/project_b_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


KeyboardInterrupt: 

## Evaluation